In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
all_tweets = pd.read_csv("data/emojis.csv")

import re
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

emoji_counts = all_tweets['emoji'].value_counts().head(20)
emoji_counts = emoji_counts.to_frame()
emoji_list = np.array(emoji_counts.index)

df = all_tweets.loc[all_tweets['emoji'].isin(emoji_list)]
pattern = r'(http://[^"\s]+)|(@\w+)|(:)|([^\w\d\s\.\?\!])'

df.loc[:, "text"] = df.text.str.replace(pattern, "").values

C:\Users\subon\Desktop\anaconda\envs\test\lib\site-packages\pandas\core\indexing.py:1047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [3]:
actual_emoji = df['emoji'].value_counts().index.values
label_emoji_mapping = dict([(label, emoji) for label, emoji in zip(range(20), actual_emoji)])
emoji_label_mapping = dict([(emoji, label) for label, emoji in label_emoji_mapping.items()])

In [4]:
from sklearn.model_selection import train_test_split


train_sentences, test_sentences, train_emojis, test_emojis = train_test_split(df['text'], df['emoji'], 
                                                                              stratify = df['emoji'], random_state = 3011)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

VOCAB_SIZE = 10000 # to prevent MemoryError
vectorizer = TfidfVectorizer(stop_words = 'english', max_features = VOCAB_SIZE)

X_train = vectorizer.fit_transform(train_sentences)
X_test = vectorizer.transform(test_sentences)

print(X_train.shape, X_test.shape)

(321638, 10000) (107213, 10000)


In [6]:
y_train, y_test = train_emojis.replace(emoji_label_mapping), test_emojis.replace(emoji_label_mapping)

In [7]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf_forest = RandomForestClassifier(n_estimators=200, oob_score=False)
clf_forest.set_params(max_features=10000)
clf_forest.fit(X_train, y_train)